# 逻辑回归模型：隐藏因子

逻辑回归模型在工业界应用十分广泛。体现了数据建模过程中很重要的思想：对问题划分层次，并利用非线性变换和线性模型的组合，将未知的复杂问题分解为已知的简单问题。
- 在模型层面上，逻辑回归是被用来解决分类问题的。由于分类是一个非线性问题，所以建模的难点在于如何把一个非线性问题转化为线性问题。相当多的模型都是围绕以下两个思路展开。
    1. 从分解问题的角度入手：通过引入隐含变量，将问题分为两个层次，一个是线性的隐含变量模型，另一个是基于隐含变量模型结果的非线性变换。
    2. 从图像的角度入手：逻辑回归是通过非线性的空间变换，将原空间内非线性的分类问题转化为新空间内的线性问题。
- 在模型评估层面，讨论了两类互相有关联的评估指标。对于分类问题的预测结果，可以定义相应的查准率（precision）、查全率（recall），以及综合这两种的。对于基于概率的分类模型，还可以绘制其ROC曲线，以及计算曲线下面积AUC。这些指标都有相应的概率解释。因此虽然它们都是定义在二元分类问题上的，但可以很自然地推广到多元分类问题以及其他场景，成为通用的评估手段。

可以毫不夸张地说：
> 理解好逻辑回归的细节，就掌握了数据建模的精髓。

## 1. 目标要求
针对二元分类问题搭建逻辑回归模型，所采用数据是美国个人收入的普查数据,来源为美国加州大学欧文分校。如何利用逻辑回归模型预测个人的年收入分类（>50k、<=50k）成为我们关注的中心。

### 1.1 数据探索分析：直观印象

In [ ]:
from __future__ import print_function
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.graphics.mosaicplot import mosaic

In [ ]:
# 读取数据
dataPath = "./data/adult.data"
data = pd.read_csv(dataPath)
data.head()

In [ ]:
# 选取特定列的数据进行后续分析
cols = ["age", "hours_per_week", "education_num", "capital_gain","capital_loss","label"]
data = data[cols]

** 利用pandas，将文本型类型变量转化为数字变量 **

在源数据的基础上新生成一个变量“label_code”,根据label的不同取值对应生成，共有两个取值：0表示“ <=50K ”，1表示“ >50K ”。

In [ ]:
data["label_code"] = pd.Categorical(data['label']).codes
data.head()

** 各变量的分布情况 **

采用直方图（histogram）将变量的分布情况可视化

In [ ]:
# 画直方图，直观了解数据
data[["age", "hours_per_week", "education_num", "label_code"]].hist(
        rwidth=0.9, figsize=(8, 8), alpha=0.6, color="grey")
plt.show()

**利用统计指标分析数据**

数据的基本统计信息，如平均值、标准差、极值等

In [ ]:
data.describe()

利用交叉报表（crosstab）来描述两个变量之间的关系

In [ ]:
# 计算education_num, label交叉报表
cross1 = pd.crosstab(pd.qcut(data["education_num"],  q=[0, .25, .5, .75, 1], precision=1), data["label"])
print("显示education_num, label交叉报表：")
print(cross1)
# 将交叉报表图形化
props = lambda key: {"color": "#aaaaaa" if ' >50K' in key else "#C6E2FF"}
mosaic(cross1[[" >50K", " <=50K"]].stack(), properties=props)
# 计算hours_per_week, label交叉报表
cross2 = pd.crosstab(pd.cut(data["hours_per_week"], 5), data["label"])
# 将交叉报表归一化，利于分析数据
cross2_norm = cross2.div(cross2.sum(1).astype(float), axis=0)
print("显示hours_per_week, label交叉报表：")
print(cross2_norm)
# 图形化归一化后的交叉报表
cross2_norm.plot(kind="bar", color=["#C6E2FF", "0.45"], rot=0)
plt.show()

根据镶嵌图可以看出： 1. 随着受教育年限的增加，年收入大于5w的比例也随之增大；

根据条形图可以看出： 1. 当工作时间小于80h，年收入大于5w的比例与工作时间成正比；当工作时间大于80h，年收入5w的比例反而下降了。

### 1.2 搭建模型

**使用第三方库Statsmodels搭建模型，并利用统计学方法分析模型参数估计值的稳定性**

In [ ]:
def trainModel(data):
    """
    搭建逻辑回归模型，并训练模型
    """
    formula = "label_code ~ age + education_num + capital_gain + capital_loss + hours_per_week"
    model = sm.Logit.from_formula(formula, data=data)
    re = model.fit()
    return re

In [ ]:
def modelSummary(re):
    """
    分析逻辑回归模型的统计性质
    """
    # 整体统计分析结果
    print(re.summary())
    # 用f test检验education_num的系数是否显著
    print("检验假设education_num的系数等于0：")
    print(re.f_test("education_num=0"))
    # 用f test检验两个假设是否同时成立
    print("检验假设education_num的系数等于0.32和hours_per_week的系数等于0.04同时成立：")
    print(re.f_test("education_num=0.32, hours_per_week=0.04"))

In [ ]:
# 将数据分为训练集和测试集
trainSet, testSet = train_test_split(data, test_size=0.2, random_state=2310)
# 训练模型并分析模型效果
model = trainModel(trainSet)
modelSummary(model)

### 1.3 理解模型结果

**模型参数的意义：1.事件的发生比；2.边际效应：变量变动对事件发生概率的影响**

In [ ]:
def interpretModel(re):
    """
    理解模型结果

    参数
    ----
    re ：BinaryResults，训练好的逻辑回归模型
    """
    conf = re.conf_int()
    conf['OR'] = re.params
    # 计算各个变量对事件发生比的影响
    # conf里面的三列，分别对应着估计值的下界、上界和估计值本身
    conf.columns = ['2.5%', '97.5%', 'OR']
    print("各个变量对事件发生比的影响：")
    print(np.exp(conf))
    # 计算各个变量的边际效应
    print("各个变量的边际效应：")
    print(re.get_margeff(at="overall").summary())

interpretModel(model)

### 1.4 评估模型效果

**查准率与查全率**

In [ ]:
def makePrediction(re, testSet, alpha=0.5):
    """
    使用训练好的模型对测试数据做预测
    """
    # 关闭pandas有关chain_assignment的警告
    pd.options.mode.chained_assignment = None
    # 计算事件发生的概率
    testSet["prob"] = re.predict(testSet)
    print("事件发生概率（预测概率）大于0.6的数据个数：")
    print(testSet[testSet["prob"] > 0.6].shape[0])  # 输出值为576
    print("事件发生概率（预测概率）大于0.5的数据个数：")
    print(testSet[testSet["prob"] > 0.5].shape[0])  # 输出值为834
    # 根据预测的概率，得出最终的预测
    testSet["pred"] = testSet.apply(lambda x: 1 if x["prob"] > alpha else 0, axis=1)
    return testSet

In [ ]:
def evaluation(re):
    """
    计算预测结果的查准查全率以及f1

    参数
    ----
    re ：DataFrame，预测结果，里面包含两列：真实值‘lable_code’、预测值‘pred’
    """
    bins = np.array([0, 0.5, 1])
    label = re["label_code"]
    pred = re["pred"]
    tn, fp, fn, tp = np.histogram2d(label, pred, bins=bins)[0].flatten()
    print('tp:',tp,'fp:',fp,'fn:',fn,'tn:',tn)
    precision = tp / (tp + fp)  # 0.951
    recall = tp / (tp + fn)  # 0.826
    f1 = 2 * precision * recall / (precision + recall)  # 0.884
    print("查准率: %.3f, 查全率: %.3f, f1: %.3f" % (precision, recall, f1))

In [ ]:
testSet = makePrediction(model, testSet)
evaluation(testSet)

**ROC曲线与AUC**

In [ ]:
from sklearn import metrics

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(testSet['label_code'], testSet['prob'])
# auc = metrics.auc(fpr,tpr)

In [ ]:
tpr[thresholds>0.5],fpr,thresholds,auc

In [ ]:
# 为在Matplotlib中显示中文，设置特殊字体
plt.rcParams["font.sans-serif"]=["SimHei"]
# 创建一个图形框
fig = plt.figure(figsize=(6, 6), dpi=80)
# 在图形框里只画一幅图
ax = fig.add_subplot(1, 1, 1)
ax.set_title("%s" % "ROC曲线".decode("utf-8"))
ax.set_xlabel("False positive rate")
ax.set_ylabel("True positive rate")
ax.plot([0, 1], [0, 1], "r--")
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.plot(fpr, tpr, "k", label="%s; %s = %0.2f" % ("ROC曲线".decode("utf-8"),"曲线下面积（AUC）".decode("utf-8"), auc))
ax.fill_between(fpr, tpr, color="grey", alpha=0.6)
legend = plt.legend(shadow=True)
plt.show()